# Lesson 3: Building an Agent Reasoning Loop

What if a user asks a complex question that requires multiple steps, or a vague question that requires clarification? In a single shot prompt setting, this becomes very 

limiting. With an agent reasoning loop, an agent is able to reason over several tools called over multiple steps.

## Setup

In [10]:
from helper import get_openai_api_key
OPENAI_API_KEY = get_openai_api_key()

In [11]:
import nest_asyncio
nest_asyncio.apply()

## Load the data

To download this paper, below is the needed code:

#!wget "https://openreview.net/pdf?id=VtmBAGCN7o" -O metagpt.pdf

**Note**: The pdf file is included with this lesson. To access it, go to the `File` menu and select`Open...`.

## Setup the Query Tools

In [12]:
from utils import get_doc_tools

vector_tool, summary_tool = get_doc_tools("metagpt.pdf", "metagpt")

In [13]:
from llama_index.llms.openai import OpenAI

llm = OpenAI(model="gpt-3.5-turbo", temperature=0)

## Setup Function Calling Agent

<div><img src="images/lesson3-1.png" style="width: 50%"/>


In [14]:
from llama_index.core.agent import FunctionCallingAgentWorker
from llama_index.core.agent import AgentRunner

agent_worker = FunctionCallingAgentWorker.from_tools(
    [vector_tool, summary_tool], 
    llm=llm, 
    verbose=True
)
agent = AgentRunner(agent_worker)

In [15]:
response = agent.query(
    "Tell me about the agent roles in MetaGPT, "
    "and then how they communicate with each other."
)

Added user message to memory: Tell me about the agent roles in MetaGPT, and then how they communicate with each other.
=== Calling Function ===
Calling function: summary_tool_metagpt with args: {"input": "agent roles in MetaGPT"}
=== Function Output ===
The agent roles in MetaGPT include Product Manager, Architect, Project Manager, Engineer, and QA Engineer. The Product Manager focuses on business-oriented analysis and insights derivation, the Architect translates requirements into system design components, the Project Manager handles task distribution, Engineers execute designated classes and functions, and the QA Engineer formulates test cases to ensure code quality. These roles work together in a structured workflow following Standard Operating Procedures (SOPs) to efficiently tackle complex tasks in software development.
=== Calling Function ===
Calling function: summary_tool_metagpt with args: {"input": "how agents communicate with each other in MetaGPT"}
=== Function Output ===
A

In [16]:
# Verify the source of this response
print(response.source_nodes[0].get_content(metadata_mode="all"))

page_label: 1
file_name: metagpt.pdf
file_path: metagpt.pdf
file_type: application/pdf
file_size: 16911937
creation_date: 2024-05-15
last_modified_date: 2024-05-11

Preprint
METAGPT: M ETA PROGRAMMING FOR A
MULTI -AGENT COLLABORATIVE FRAMEWORK
Sirui Hong1∗, Mingchen Zhuge2∗, Jonathan Chen1, Xiawu Zheng3, Yuheng Cheng4,
Ceyao Zhang4,Jinlin Wang1,Zili Wang ,Steven Ka Shing Yau5,Zijuan Lin4,
Liyang Zhou6,Chenyu Ran1,Lingfeng Xiao1,7,Chenglin Wu1†,J¨urgen Schmidhuber2,8
1DeepWisdom,2AI Initiative, King Abdullah University of Science and Technology,
3Xiamen University,4The Chinese University of Hong Kong, Shenzhen,
5Nanjing University,6University of Pennsylvania,
7University of California, Berkeley,8The Swiss AI Lab IDSIA/USI/SUPSI
ABSTRACT
Remarkable progress has been made on automated problem solving through so-
cieties of agents based on large language models (LLMs). Existing LLM-based
multi-agent systems can already solve simple dialogue tasks. Solutions to more
complex tasks, however, 

<div><img src="images/lesson3-2.png" style="width: 50%"/>

In [17]:
response = agent.chat(
    "Tell me about the evaluation datasets used."
)

Added user message to memory: Tell me about the evaluation datasets used.
=== Calling Function ===
Calling function: summary_tool_metagpt with args: {"input": "evaluation datasets used in MetaGPT"}
=== Function Output ===
The evaluation datasets used in MetaGPT include HumanEval, MBPP, SoftwareDev, and humaneval-x.
=== LLM Response ===
The evaluation datasets used in MetaGPT include HumanEval, MBPP, SoftwareDev, and humaneval-x.


In [18]:
response = agent.chat("Tell me the results over one of the above datasets.")

Added user message to memory: Tell me the results over one of the above datasets.
=== Calling Function ===
Calling function: vector_tool_metagpt with args: {"query": "results over HumanEval dataset", "page_numbers": ["7"]}
=== Function Output ===
MetaGPT achieved 85.9% and 87.7% Pass rates over the HumanEval dataset.
=== LLM Response ===
MetaGPT achieved 85.9% and 87.7% Pass rates over the HumanEval dataset.


## Lower-Level: Debuggability and Control

## Key Benefits of Agent Control
1. *Decoupling of Task Creation and Execution*
    - users gain the flexibility to schedule task execution according to their needs
2. *Enhance debugging*
    - offers deep insight into every step of the execution process, improving troubleshooting capabilities
3. *Steerability*
    - allows users to directly modify intermediate steps and incorporate human feedback for refined control

In [19]:
agent_worker = FunctionCallingAgentWorker.from_tools(
    [vector_tool, summary_tool], 
    llm=llm, 
    verbose=True
)
agent = AgentRunner(agent_worker)

In [20]:
task = agent.create_task(
    "Tell me about the agent roles in MetaGPT, "
    "and then how they communicate with each other."
)

In [21]:
step_output = agent.run_step(task.task_id)

Added user message to memory: Tell me about the agent roles in MetaGPT, and then how they communicate with each other.
=== Calling Function ===
Calling function: summary_tool_metagpt with args: {"input": "agent roles in MetaGPT"}


Retrying llama_index.llms.openai.base.OpenAI._achat in 0.3266696415083816 seconds as it raised RateLimitError: Error code: 429 - {'error': {'message': 'Rate limit reached for gpt-3.5-turbo in organization org-vqFYxgzzHIM2dUljWi76qehk on tokens per min (TPM): Limit 60000, Used 59654, Requested 4041. Please try again in 3.695s. Visit https://platform.openai.com/account/rate-limits to learn more.', 'type': 'tokens', 'param': None, 'code': 'rate_limit_exceeded'}}.
Retrying llama_index.llms.openai.base.OpenAI._achat in 0.04244013960077708 seconds as it raised RateLimitError: Error code: 429 - {'error': {'message': 'Rate limit reached for gpt-3.5-turbo in organization org-vqFYxgzzHIM2dUljWi76qehk on tokens per min (TPM): Limit 60000, Used 59406, Requested 4270. Please try again in 3.676s. Visit https://platform.openai.com/account/rate-limits to learn more.', 'type': 'tokens', 'param': None, 'code': 'rate_limit_exceeded'}}.


=== Function Output ===
The agent roles in MetaGPT include Product Manager, Architect, Project Manager, Engineer, Developer, Tester, QA Engineer, experiment conductors, module designers, write-up leaders, contributors to experiments and methods, guidance providers, evaluation metric designers, individuals who initiated the project, made significant code contributions, and provided overall advice and support. Each role has specific responsibilities and expertise tailored to different aspects of the collaborative software development process within the MetaGPT framework.


In [22]:
completed_steps = agent.get_completed_steps(task.task_id)
print(f"Num completed for task {task.task_id}: {len(completed_steps)}")
print(completed_steps[0].output.sources[0].raw_output)

Num completed for task 99875fbc-8273-4668-b5bd-5034f9e74f90: 1
The agent roles in MetaGPT include Product Manager, Architect, Project Manager, Engineer, Developer, Tester, QA Engineer, experiment conductors, module designers, write-up leaders, contributors to experiments and methods, guidance providers, evaluation metric designers, individuals who initiated the project, made significant code contributions, and provided overall advice and support. Each role has specific responsibilities and expertise tailored to different aspects of the collaborative software development process within the MetaGPT framework.


In [23]:
upcoming_steps = agent.get_upcoming_steps(task.task_id)
print(f"Num upcoming steps for task {task.task_id}: {len(upcoming_steps)}")
upcoming_steps[0]

Num upcoming steps for task 99875fbc-8273-4668-b5bd-5034f9e74f90: 1


TaskStep(task_id='99875fbc-8273-4668-b5bd-5034f9e74f90', step_id='a3db6578-4d63-489c-8454-35ad0f5ee021', input=None, step_state={}, next_steps={}, prev_steps={}, is_ready=True)

In [24]:
step_output = agent.run_step(
    task.task_id, input="What about how agents share information?"
)

Added user message to memory: What about how agents share information?
=== Calling Function ===
Calling function: summary_tool_metagpt with args: {"input": "how agents share information in MetaGPT"}


Retrying llama_index.llms.openai.base.OpenAI._achat in 0.30496476921028537 seconds as it raised RateLimitError: Error code: 429 - {'error': {'message': 'Rate limit reached for gpt-3.5-turbo in organization org-vqFYxgzzHIM2dUljWi76qehk on tokens per min (TPM): Limit 60000, Used 59612, Requested 3969. Please try again in 3.581s. Visit https://platform.openai.com/account/rate-limits to learn more.', 'type': 'tokens', 'param': None, 'code': 'rate_limit_exceeded'}}.
Retrying llama_index.llms.openai.base.OpenAI._achat in 0.8868677274715802 seconds as it raised RateLimitError: Error code: 429 - {'error': {'message': 'Rate limit reached for gpt-3.5-turbo in organization org-vqFYxgzzHIM2dUljWi76qehk on tokens per min (TPM): Limit 60000, Used 59437, Requested 4046. Please try again in 3.483s. Visit https://platform.openai.com/account/rate-limits to learn more.', 'type': 'tokens', 'param': None, 'code': 'rate_limit_exceeded'}}.
Retrying llama_index.llms.openai.base.OpenAI._achat in 0.588675009372

=== Function Output ===
Agents in MetaGPT share information through a structured communication protocol that includes a shared message pool. This pool allows agents to publish structured messages and subscribe to relevant messages based on their profiles. By utilizing this communication interface, agents can efficiently exchange directional information from other roles and public information from the environment. Additionally, MetaGPT implements a publish-subscribe mechanism that enables agents to access important observations stored in the message pool, which can trigger actions or aid in completing tasks effectively. The shared message pool and subscription mechanism in MetaGPT facilitate efficient information sharing among agents within the simulated software company framework.


In [25]:
step_output = agent.run_step(task.task_id)
print(step_output.is_last)

=== LLM Response ===
Agents in MetaGPT share information through a structured communication protocol that includes a shared message pool. This pool allows agents to publish structured messages and subscribe to relevant messages based on their profiles. By utilizing this communication interface, agents can efficiently exchange directional information from other roles and public information from the environment. Additionally, MetaGPT implements a publish-subscribe mechanism that enables agents to access important observations stored in the message pool, which can trigger actions or aid in completing tasks effectively. The shared message pool and subscription mechanism in MetaGPT facilitate efficient information sharing among agents within the simulated software company framework.
True


In [26]:
response = agent.finalize_response(task.task_id)

In [27]:
print(str(response))

assistant: Agents in MetaGPT share information through a structured communication protocol that includes a shared message pool. This pool allows agents to publish structured messages and subscribe to relevant messages based on their profiles. By utilizing this communication interface, agents can efficiently exchange directional information from other roles and public information from the environment. Additionally, MetaGPT implements a publish-subscribe mechanism that enables agents to access important observations stored in the message pool, which can trigger actions or aid in completing tasks effectively. The shared message pool and subscription mechanism in MetaGPT facilitate efficient information sharing among agents within the simulated software company framework.
